In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime

In [2]:
#File with URLs
urls_file = "websites_speeches_pqs.csv"

In [3]:
titles = []
dates = []
contents = []

In [4]:
def get_article_title(soup):
    #Select relevant CSS pattern
    article_title = soup.select('div[class="article-title"] > h1[class="hide-mobile"]')
    #Return title text
    return article_title[0].get_text().strip()

In [5]:
def get_article_date(soup):
    #Select relevant CSS pattern
    article_date = soup.select('div[class="info-date"]')
    #Obtain date
    date = article_date[0].get_text().strip()
    #Convert date to datetime format
    return datetime.strptime(date, '%d %b %Y')

In [6]:
def get_article_content(soup):
    #Initiate empty string
    string = ''
    
    #Select relevant CSS pattern
    article_content = soup.select('div[class*="article-content"]')
    
    #Extract all article text and combine into the same string
    for con in article_content:
        string += con.get_text()
    
    return string

In [7]:
#Read URL csv
urls = pd.read_csv(urls_file)['URLs']

#Extract content from each url
for url in urls:
    page = requests.get(url)
    soup = bs(page.content, features="html.parser")
    
    titles.append(get_article_title(soup))
    dates.append(get_article_date(soup))
    contents.append(get_article_content(soup))

In [8]:
#Save to CSV
scraped_df = pd.DataFrame(zip(titles, dates, contents), columns = ['Title', 'Date', 'Content'])
scraped_df.to_csv("MTI_speeches_PQs_scraped.csv")